<a href="https://colab.research.google.com/github/dsivov/netIndex/blob/master/tensor_lattice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install -q tensorflow-lattice

In [0]:
import numpy
import random
import string
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from absl import app
from absl import flags

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
import tensorflow_lattice as tfl

flags_dict = FLAGS._flags()    
keys_list = [keys for keys in flags_dict]    
for keys in keys_list:
      FLAGS.__delattr__(keys)
FLAGS = flags.FLAGS
flags.DEFINE_integer('num_epochs', 200, 'Number of training epoch.')


Generate 4-char string and put it into numpy. Target is position

In [151]:
keys = numpy.empty((0,5), float)

keys_list=[]
for i in range(1000):
    keys_list.append(''.join(random.choices(string.ascii_uppercase +
                                 string.digits + string.ascii_lowercase, k=4)))
    
keys_list.sort()
print(keys_list)

for i in range(1000):
    key = keys_list[i]
    key_array = [ord(c)/255 for c in key]
    # add position 
    key_array.append(i/1000)
    #print (key_array)
    keys = numpy.append(keys, numpy.array([key_array]), axis=0)

   

['03W4', '03pQ', '04VZ', '07ll', '08Ri', '0CDt', '0EM2', '0NNW', '0NvL', '0TDS', '0VU5', '0VhR', '0WK0', '0dqH', '0mtt', '0nAY', '0oPe', '0quy', '1F0n', '1FQh', '1JUs', '1LMM', '1PA5', '1UEs', '1aeL', '1e2x', '1lhs', '1sK4', '22lI', '22xd', '29Ny', '2Iru', '2JJK', '2KXT', '2cR1', '2g76', '2gu4', '2mdF', '2n6V', '2phb', '2qo5', '2sI5', '2sJC', '2yLD', '2zra', '3HlP', '3Oeg', '3Oin', '3V7M', '3a3g', '3cJK', '3jo0', '3lGi', '440A', '459G', '48oq', '4QMs', '4S56', '4XbH', '4Z0t', '4bef', '4fY8', '4iaZ', '4j9x', '4qwi', '4tGW', '4zGZ', '50sj', '54Jx', '57aG', '5AWO', '5INj', '5Iwl', '5LKf', '5Nxx', '5T5Y', '5UwS', '5XPU', '5jKh', '5sl8', '5uPS', '62lE', '68Hz', '6E8I', '6QjS', '6W6T', '6Zpr', '6cmr', '6gC8', '6jKL', '6k4Y', '6tbQ', '6v1Y', '6yJ5', '75Kl', '75zo', '77iN', '7A0g', '7AtH', '7CUa', '7Ek5', '7HPn', '7Jh1', '7QrC', '7SKU', '7SmO', '7V1p', '7ZSu', '7c3g', '7e26', '7mw2', '7naa', '7qhx', '7vJh', '7vZ4', '7wLZ', '7ybS', '7yq3', '81R1', '8BQc', '8HpV', '8Im2', '8NP1', '8Ogm', '8QmJ',

Create data frame

In [152]:
df = pd.DataFrame(data=keys, columns=["X1", "X2","X3","X4","target"])
print(df)
combined_calibrators = tfl.layers.ParallelCombination()


           X1        X2        X3        X4  target
0    0.188235  0.200000  0.341176  0.203922   0.000
1    0.188235  0.200000  0.439216  0.317647   0.001
2    0.188235  0.203922  0.337255  0.352941   0.002
3    0.188235  0.215686  0.423529  0.423529   0.003
4    0.188235  0.219608  0.321569  0.411765   0.004
..        ...       ...       ...       ...     ...
995  0.478431  0.419608  0.352941  0.254902   0.995
996  0.478431  0.419608  0.407843  0.388235   0.996
997  0.478431  0.466667  0.443137  0.294118   0.997
998  0.478431  0.466667  0.458824  0.309804   0.998
999  0.478431  0.478431  0.188235  0.427451   0.999

[1000 rows x 5 columns]


Define "monotonicy" on each X, from 0 to 1

In [0]:
calibrator = tfl.layers.PWLCalibration(
 
input_keypoints=np.linspace(df['X1'].min(),
                                  df['X1'].max(),
                                  num=10),
    
dtype=tf.float32,     
output_min=0.0,
output_max=lattice_sizes[0] - 1.0,
kernel_initializer='equal_slopes',
monotonicity='increasing')
combined_calibrators.append(calibrator)

In [0]:
calibrator = tfl.layers.PWLCalibration(
 
input_keypoints=np.linspace(df['X2'].min(),
                                  df['X2'].max(),
                                  num=10),
    
dtype=tf.float32,     
output_min=0.0,
output_max=lattice_sizes[0] - 1.0,
kernel_initializer='equal_slopes',
monotonicity='increasing')
combined_calibrators.append(calibrator)

In [0]:
calibrator = tfl.layers.PWLCalibration(
 
input_keypoints=np.linspace(df['X3'].min(),
                                  df['X3'].max(),
                                  num=10),
    
dtype=tf.float32,     
output_min=0.0,
output_max=lattice_sizes[0] - 1.0,
kernel_initializer='equal_slopes',
monotonicity='increasing')
combined_calibrators.append(calibrator)

In [0]:
calibrator = tfl.layers.PWLCalibration(
 
input_keypoints=np.linspace(df['X4'].min(),
                                  df['X4'].max(),
                                  num=10),
    
dtype=tf.float32,     
output_min=0.0,
output_max=lattice_sizes[0] - 1.0,
kernel_initializer='equal_slopes',
monotonicity='increasing')
combined_calibrators.append(calibrator)

Laticce model

In [0]:
lattice_sizes = [2, 2, 2, 2]
lattice = tfl.layers.Lattice(
      lattice_sizes=lattice_sizes,
      monotonicities=['increasing','increasing', 'increasing',
                      'increasing'],
      output_min=0.0,
      output_max=1.0)

In [166]:
model = keras.models.Sequential()
model.add(combined_calibrators)
model.add(lattice)
model.compile(loss=keras.losses.mean_squared_error,
                optimizer=keras.optimizers.Adagrad(learning_rate=1.0))

features = df[['X1', 'X2', 'X3','X4']].values.astype(np.float32)
target = df[['target']].values.astype(np.float32)

model.fit(features,
  target,
  batch_size=32,
  epochs=200,
  validation_split=0.2,
  shuffle=False)


Train on 800 samples, validate on 200 samples
Epoch 1/200
800/800 [==============================] - 2s 2ms/sample - loss: 0.0104 - val_loss: 0.1380
Epoch 2/200
800/800 [==============================] - 0s 113us/sample - loss: 0.0210 - val_loss: 0.0290
Epoch 3/200
800/800 [==============================] - 0s 105us/sample - loss: 0.0137 - val_loss: 0.0280
Epoch 4/200
800/800 [==============================] - 0s 104us/sample - loss: 0.0135 - val_loss: 0.0318
Epoch 5/200
800/800 [==============================] - 0s 104us/sample - loss: 0.0121 - val_loss: 0.0259
Epoch 6/200
800/800 [==============================] - 0s 108us/sample - loss: 0.0121 - val_loss: 0.0322
Epoch 7/200
800/800 [==============================] - 0s 100us/sample - loss: 0.0149 - val_loss: 0.0484
Epoch 8/200
800/800 [==============================] - 0s 109us/sample - loss: 0.0116 - val_loss: 0.0508
Epoch 9/200
800/800 [==============================] - 0s 105us/sample - loss: 0.0118 - val_loss: 0.0332
Epoch 10/20

Predict for each string

In [159]:
weights = model.layers[0].get_weights()
print (weights)

[array([[0.0000000e+00],
       [3.9668623e-02],
       [0.0000000e+00],
       [1.4431199e-01],
       [1.7569473e-01],
       [7.0991158e-02],
       [8.4668398e-05],
       [3.7979960e-02],
       [1.3796511e-01],
       [0.0000000e+00]], dtype=float32), array([[0.03911368],
       [0.0320022 ],
       [0.01916919],
       [0.0163827 ],
       [0.07459845],
       [0.12360865],
       [0.18055484],
       [0.26325846],
       [0.15318221],
       [0.03764838]], dtype=float32), array([[0.19353238],
       [0.12274989],
       [0.0552654 ],
       [0.04045397],
       [0.02678332],
       [0.08892113],
       [0.08745855],
       [0.04790837],
       [0.03588909],
       [0.02295196]], dtype=float32), array([[1.9711494e-01],
       [1.6953975e-01],
       [1.3340110e-01],
       [4.1132927e-02],
       [1.5532970e-03],
       [1.3999343e-03],
       [3.3503771e-03],
       [1.3489723e-03],
       [1.0634065e-03],
       [8.2015991e-05]], dtype=float32)]


In [167]:
predictions = model.predict(features[:1000])
#print(target[:100])
#print(features[:100])
print(predictions)

[[0.05409437]
 [0.05440687]
 [0.05426188]
 [0.05450696]
 [0.05432122]
 [0.05477293]
 [0.05465604]
 [0.0550123 ]
 [0.05527324]
 [0.05536845]
 [0.05555055]
 [0.05580787]
 [0.055525  ]
 [0.0578505 ]
 [0.05934871]
 [0.05936579]
 [0.05953797]
 [0.05993696]
 [0.06796996]
 [0.06814678]
 [0.06828134]
 [0.06822404]
 [0.06811881]
 [0.06873628]
 [0.07053341]
 [0.07102244]
 [0.07238044]
 [0.07316661]
 [0.08088252]
 [0.08095875]
 [0.08087267]
 [0.08169601]
 [0.08143058]
 [0.08158638]
 [0.08384341]
 [0.08453498]
 [0.08473225]
 [0.08570142]
 [0.08574156]
 [0.08615381]
 [0.08624098]
 [0.08635019]
 [0.0864048 ]
 [0.08691466]
 [0.0870858 ]
 [0.09490202]
 [0.09500143]
 [0.0950127 ]
 [0.09530073]
 [0.09677613]
 [0.09716514]
 [0.09839804]
 [0.09867106]
 [0.10700874]
 [0.10720297]
 [0.10760216]
 [0.10810333]
 [0.10805985]
 [0.10896027]
 [0.10893486]
 [0.11037951]
 [0.11090925]
 [0.11153223]
 [0.11158039]
 [0.11269812]
 [0.11287943]
 [0.11336853]
 [0.12067942]
 [0.12050429]
 [0.12076083]
 [0.12114264]
 [0.12